In [1]:
import sys
import IPython
import numpy as np
import pandas as pd
import sklearn as sk

print ('Python version: %s.%s.%s' % sys.version_info[:3])
print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('pandas version:', pd.__version__)
print ('scikit-learn version:', sk.__version__)
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
import feather
## Usando OneHotEncoder
from sklearn.preprocessing import LabelEncoder


#walmart = feather.read_dataframe('../data/transformed_data.feather')
walmart = pd.read_csv('../data/train.csv')
walmart.head()

X = walmart.iloc[:, walmart.columns != 'TripType']
y = walmart.iloc[:, walmart.columns=="TripType"]
label_encoder = LabelEncoder()  ## Para convertir a enteros
label_encoder2 = LabelEncoder()  ## Para convertir a enteros
label_encoder3 = LabelEncoder()  ## Para convertir a enteros

## Convertirmos a enteros, i.e. setosa -> 0, etc
y = label_encoder.fit_transform(y)
print(X)
X['Weekday_e'] = label_encoder2.fit_transform(X['Weekday'])
X['DepartmentDescription_e'] = label_encoder3.fit_transform(X['DepartmentDescription'].map(lambda x: str(x)))
X['VisitNumber'] = X['VisitNumber'].astype(int)

X = X[['VisitNumber', 'Weekday_e', 'ScanCount', 'DepartmentDescription_e']]


X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3)



Python version: 3.6.3
IPython version: 6.2.1
numpy version: 1.13.3
pandas version: 0.21.0
scikit-learn version: 0.19.1


/Users/timgivois/personal/venv3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/timgivois/personal/venv3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/timgivois/personal/venv3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usi

        VisitNumber Weekday           Upc  ScanCount  \
0                 5  Friday  6.811315e+10         -1   
1                 7  Friday  6.053882e+10          1   
2                 7  Friday  7.410811e+09          1   
3                 8  Friday  2.238404e+09          2   
4                 8  Friday  2.006614e+09          2   
5                 8  Friday  2.006619e+09          2   
6                 8  Friday  2.006614e+09          1   
7                 8  Friday  7.004803e+09          1   
8                 8  Friday  2.238495e+09          1   
9                 8  Friday  2.238400e+09         -1   
10                8  Friday  5.200010e+09          1   
11                8  Friday  8.867930e+10          2   
12                8  Friday  2.200600e+10          1   
13                8  Friday  2.236760e+09          1   
14                8  Friday  8.867930e+10         -1   
15                8  Friday  2.238400e+09          2   
16                8  Friday  3.019294e+09       

In [2]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model_xg = XGBClassifier()
% time model_xg.fit(X_train, y_train)



CPU times: user 8min 31s, sys: 3.64 s, total: 8min 34s
Wall time: 2min 47s


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [3]:
y_pred = model_xg.predict(X_train)
predictions = [round(value) for value in y_pred]
classification_report(y_train, predictions)
print(accuracy_score(y_train, predictions))

0.352143896392


In [6]:
#walmart_test = feather.read_dataframe('../data/test_transformed_data.feather')
walmart_test = pd.read_csv('../data/test.csv')
walmart_test.head()
X2 = walmart_test

X2['Weekday_e'] = label_encoder2.fit_transform(X2['Weekday'])
X2['DepartmentDescription_e'] = label_encoder3.fit_transform(X2['DepartmentDescription'].map(lambda x: str(x)))
X2['VisitNumber'] = X2['VisitNumber'].astype(int)
X2 = X2[['VisitNumber', 'Weekday_e', 'ScanCount', 'DepartmentDescription_e']]



In [8]:

y_pred = model_xg.predict(X2)
predictions = [round(value) for value in y_pred]
y_final = label_encoder.inverse_transform(predictions)
X2['TripType'] = y_final
data4 = X2[['VisitNumber','TripType']].drop_duplicates(["VisitNumber", "TripType"])
data_final = pd.get_dummies(data4, columns = ['TripType']).groupby(["VisitNumber"]).mean().round()

for each in data_final:
    data_final[each] = data_final[each].astype(int)



ValueError: feature_names mismatch: ['VisitNumber', 'Weekday_e', 'ScanCount', 'DepartmentDescription_e'] ['VisitNumber', 'Weekday_e', 'ScanCount', 'DepartmentDescription_e', 'TripType']
training data did not have the following fields: TripType

In [11]:
data_sample = pd.read_csv('../data/sample_submission.csv')
data_final['TripType_4'] = np.zeros(len(data_final))
data_final['TripType_12'] = np.zeros(len(data_final))
data_final.to_csv('../data/fourth_submisson.csv')
data_final.head()

,TripType_3,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_14,TripType_15,TripType_18,TripType_19,...,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999,TripType_4,TripType_12
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
